In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
model = 'geneva_200m'
params_file = 'swirl_03'
output_folder = r'../Outputs'

lvl0_filename = f'{model}_{params_file}_day1_lvl0.csv'
output_filename = f'{model}_{params_file}_day1_lvl1.csv'

# Import catalogue lvl0

In [3]:
level0_csv_path = os.path.join(output_folder, lvl0_filename)
level0_data = pd.read_csv(level0_csv_path)

# Create catalogue lvl1

In [4]:
# Function to identify the different depths of an eddy
def identify_eddy(level0_data, idx_already_aggregated, id_eddy, dist_threshold, depth_threshold):
    mask = (
            (~level0_data['id'].isin(idx_already_aggregated)) &
            (level0_data['date'] == level0_data.at[id_eddy, 'date']) &
            (level0_data['rotation_direction'] == level0_data.at[id_eddy, 'rotation_direction']) &
            (np.sqrt((level0_data['xc'] - level0_data.at[id_eddy, 'xc'])**2 + (level0_data['yc'] - level0_data.at[id_eddy, 'yc'])**2) < dist_threshold)
        )
    
    filtered_eddies = level0_data.loc[mask]
    sorted_by_depth = filtered_eddies.sort_values('depth_[m]', ascending=False).reset_index(drop=False)
    
    for i in range(1,len(sorted_by_depth)):
        if sorted_by_depth.iloc[i]['depth_index'] - sorted_by_depth.iloc[i-1]['depth_index']> depth_threshold:
            drop_idxs = sorted_by_depth.index[i:]
            sorted_by_depth = sorted_by_depth.drop(index=drop_idxs)            
            break
    
    return sorted_by_depth

In [7]:
# Parameters
dist_threshold = 5  # in number of cells
depth_threshold = 5 # in number of cells
eddy_rows_lvl1 = []  # Collect aggregated rows here
id_level1 = 0
idx_already_aggregated = set()

# Main loop
for idx in level0_data['id']: #level0_data['id']
    if idx in idx_already_aggregated:
        continue
        
    aggregated_data = identify_eddy(level0_data, idx_already_aggregated, idx, dist_threshold, depth_threshold)

    row = {
        'id': id_level1,
        'id_lvl0': aggregated_data['id'].tolist(),
        'time_index': aggregated_data.at[0, 'time_index'],
        'date': aggregated_data.at[0, 'date'],
        'depth_min_[m]': aggregated_data['depth_[m]'].min(),
        'depth_max_[m]': aggregated_data['depth_[m]'].max(),
        'xc_mean': aggregated_data['xc'].mean(),
        'yc_mean': aggregated_data['yc'].mean(),
        'surface_area_mean_[m2]': aggregated_data['surface_area_[m2]'].mean(),
        'volume_[m3]': aggregated_data['volume_slice_[m3]'].sum(),
        'rotation_direction': aggregated_data.at[0, 'rotation_direction'],
        'kinetic_energy_[MJ]': aggregated_data['kinetic_energy_[MJ]'].sum()
    }

    eddy_rows_lvl1.append(row)
    idx_already_aggregated.update(aggregated_data['id'].tolist())
    id_level1 += 1

# Create the final DataFrame using pd.concat
df_catalogue_level1 = pd.concat([pd.DataFrame([row]) for row in eddy_rows_lvl1], ignore_index=True)

# Save 

In [8]:
df_catalogue_level1.to_csv(os.path.join(output_folder, output_filename), index=False)

# Plot

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
level0_data.iloc[df_catalogue_level1['id_lvl0'].iloc[0]]

In [ ]:
df_catalogue_level1[df_catalogue_level1['date']==df_catalogue_level1['date'][0]]

In [ ]:
snapshot_catalogue_lvl1 = df_catalogue_level1[df_catalogue_level1['date']==df_catalogue_level1['date'][0]]
plt.scatter(snapshot_catalogue_lvl1['xc_mean'], snapshot_catalogue_lvl1['yc_mean'], alpha=0.5, s=2)